This notebook uses a set of functions and module within the bispy package.

GAP records are retrieved and cached based on valid/accepted ITIS species names and the invalid/not accepted ITIS species names.

In [1]:
#Import needed packages
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed

gap = bispy.gap.Gap()
bis_utils = bispy.bis.Utils()

In [2]:
# Open up the cached ITIS species list from Consult with ITIS.ipynb
with open("cache/itis.json", "r") as f:
    wlci_species = json.loads(f.read())

In [3]:
#Create list containing sciencitific names, ITIS usage, kingdom, and biological taxonomy
itis_explore = list()
for itis_doc_set in wlci_species:
    for itis_doc in itis_doc_set["itis_data"]:
        itis_explore.append({"scientific_name": itis_doc["nameWInd"],'itis_usage':itis_doc['usage'],"kingdom": itis_doc["kingdom"], 'biological_taxonomy': itis_doc["biological_taxonomy"]})     

In [4]:
# Gap only models mammals, birds, reptiles, and amphibians. Create list restricted the kingdom Animalia
possible_gap_species=[e for e in itis_explore if e['kingdom'] not in ['Plantae']]

In [5]:
#Create list of kingdom Animalia ITIS accepted/valid species names
valid_species_names=[e for e in possible_gap_species if e['scientific_name'] not in {'Amphispiza belli','Thrichomys fosteri','Tetrao tetrix','Brucella abortus','Equus burchelli','Bassia prostrata'}]

In [6]:
valid_species_names

[{'scientific_name': 'Ursus americanus',
  'itis_usage': 'valid',
  'kingdom': 'Animalia',
  'biological_taxonomy': [{'rank': 'Kingdom', 'name': 'Animalia'},
   {'rank': 'Subkingdom', 'name': 'Bilateria'},
   {'rank': 'Infrakingdom', 'name': 'Deuterostomia'},
   {'rank': 'Phylum', 'name': 'Chordata'},
   {'rank': 'Subphylum', 'name': 'Vertebrata'},
   {'rank': 'Infraphylum', 'name': 'Gnathostomata'},
   {'rank': 'Superclass', 'name': 'Tetrapoda'},
   {'rank': 'Class', 'name': 'Mammalia'},
   {'rank': 'Subclass', 'name': 'Theria'},
   {'rank': 'Infraclass', 'name': 'Eutheria'},
   {'rank': 'Order', 'name': 'Carnivora'},
   {'rank': 'Suborder', 'name': 'Caniformia'},
   {'rank': 'Family', 'name': 'Ursidae'},
   {'rank': 'Genus', 'name': 'Ursus'},
   {'rank': 'Species', 'name': 'Ursus americanus'}]},
 {'scientific_name': 'Tamiasciurus hudsonicus',
  'itis_usage': 'valid',
  'kingdom': 'Animalia',
  'biological_taxonomy': [{'rank': 'Kingdom', 'name': 'Animalia'},
   {'rank': 'Subkingdom', 

In [7]:
# Use joblib to run multiple requests for GAP records in parallel via kingdom Animalia valid/accepted scientific names
gap_results = Parallel(n_jobs=8)(delayed(gap.gap_species_search)(r["scientific_name"]) for r in valid_species_names)
len(gap_results)

69

In [9]:
# Filter to give just cases where GAP species names matched with kingdom Animalia valid/accepted ITIS names
gap_success=[i for i in gap_results if i['processing_metadata']['status'] == 'success']
len(gap_success)

40

In [10]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/gap.json", gap_success))

{'Doc Cache File': 'cache/gap.json',
 'Number of Documents in Cache': 40,
 'Document Number 26': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-09T19:49:33.822883',
   'status_message': 'Exact Match',
   'api': "https://www.sciencebase.gov/catalog/items?parentId=527d0a83e4b0850ea0518326&format=json&fields=identifiers,files,webLinks,distributionLinks,dates&filter=itemIdentifier%3D{'key': 'Falco sparverius'}"},
  'parameters': {'Scientific Name': 'Falco sparverius'},
  'GAP Species': {'GAP Habitat Map Item': 'https://www.sciencebase.gov/catalog/item/58fa4487e4b0b7ea54524c50',
   'GAP Range Map Item': 'https://www.sciencebase.gov/catalog/item/59f5ec58e4b063d5d307e64d',
   'GAP Habitat Map WMS': 'https://www.sciencebase.gov/geoserver/CONUS_HabMap_2001/wms?service=WMS&version=1.1.0&request=GetCapabilities',
   'GAP Modeling Database Parameters URL': 'https://www.sciencebase.gov/catalog/file/get/58fa4487e4b0b7ea54524c50?f=__disk__ab%2F18%2F3a%2Fab183a4bcae9664565

Check to see if any invalid/not accepted ITIS specie names matched with GAP records

In [ ]:
#List of kingdom Animalia invalid/not accepted ITIS species names
invalid_species_name=[i for i in possible_gap_species if i['itis_usage'] == 'invalid']
invalid_species_name.extend([i for i in possible_gap_species if i['itis_usage'] == 'not accepted'])

In [ ]:
# Use joblib to run multiple requests for GAP records in parallel via kingdom Animalia invalid/not accepted ITIS scientific names
invalid_gap_results = Parallel(n_jobs=8)(delayed(gap.gap_species_search)(r["scientific_name"]) for r in invalid_species_name)
#len(gap_results)

In [ ]:
# Filter to give just cases where GAP species names matched with kingdom Animalia invalid/not accepted ITIS species names
invalid_result=[i for i in invalid_gap_results if i['processing_metadata']['status'] == 'success']
len(invalid_result)

In [ ]:
#Display result to identify the GAP record that matched the invalid/not accepted ITIS name
invalid_result

In [ ]:
# Cache the retrieved document and return/display a sample for verification
display(bis_utils.doc_cache("cache/Results of Consultations with Invalid ITIS Species Names/invalid_gap.json", invalid_result))